<a href="https://colab.research.google.com/github/FatemehMomeni/AdvDM_pro1/blob/main/DM_pro1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
from sklearn import preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Showing dataframe completely

In [ ]:
pd.set_option('display.width', 2000)
pd.set_option('display.max_columns', 61)
pd.set_option('display.max_rows', 200)

NameError: ignored

# Reading csv files

In [ ]:
questions = pd.read_csv('/content/DM1_dataset/Questions.csv')
answers = pd.read_csv('/content/DM1_dataset/Answers.csv')

# Missing values

## calculating percentage

In [ ]:
#counting number of missing values per column
NaN_num = answers.isna().sum()

#define a dictionary for storing percentage of missing values in each column
NaN_percent = dict()
answers_columns = answers.columns
total_num = len(answers)

#calculating percentage of missing values per column
for num in range(len(NaN_num)):
    NaN_percent[answers_columns[num]] = NaN_num[num] * 100 / total_num

## Handling

In [ ]:
#sorting list of percentages to find columns with max number of missing values that stored in max_miss list
sort = sorted(NaN_percent.items(), key=lambda x: x[1], reverse=True)
max_miss = list()
for i in sort:
    if i[1] >= 40.0:
        max_miss.append(i[0])
    else:
        break

In [ ]:
# deleting columns have many missing values
processed_ans = answers.drop(axis=1, columns=max_miss)

In [ ]:
# finding type of columns
print(processed_ans.dtypes)

# filling missing values with mean for numeric(float64) columns
numeric = ['Age', 'WorkWeekHrs']
for i in numeric:
    processed_ans[i] = processed_ans[i].fillna(processed_ans[i].mean())

# filling missing values with mode for categorical columns
categorical = ['MainBranch', 'Hobbyist', 'Age1stCode', 'CompFreq', 'Country', 'CurrencyDesc', 'CurrencySymbol',
               'EdLevel', 'Employment', 'Ethnicity', 'Gender', 'JobSat', 'JobSeek', 'NEWDevOps', 'NEWDevOpsImpt',
               'NEWEdImpt', 'NEWLearn', 'NEWOffTopic', 'NEWOnboardGood', 'NEWOtherComms', 'NEWOvertime', 'NEWPurpleLink',
               'OpSys', 'OrgSize', 'PurchaseWhat', 'Sexuality', 'SOAccount', 'SOComm', 'SOPartFreq', 'SOVisitFreq',
               'SurveyEase', 'SurveyLength', 'Trans', 'UndergradMajor', 'WelcomeChange', 'YearsCode', 'YearsCodePro']
for i in categorical:
    processed_ans[i] = processed_ans[i].fillna(processed_ans[i].mode()[0])


In [ ]:
# remained columns
remained = list()
for i in answers_columns:
    if i not in (max_miss + numeric + categorical):
        remained.append(i)

multi_value = dict()
for i in range(1, len(remained)):
    multi_value.clear()
    column = processed_ans[remained[i]]
    for row in column:
        values = str(row).split(';')
        for val in values:
            if val != 'nan':
                if val not in multi_value.keys():
                    multi_value[val] = 1
                else:
                    multi_value[val] += 1
    if remained[i] == 'LanguageWorkedWith':
        LanguageWorkedWith = multi_value.keys()
    fill_value = max(multi_value, key=multi_value.get)
    processed_ans[remained[i]] = processed_ans[remained[i]].fillna(fill_value)

# Outliers

In [ ]:
# use label encoder to encode object data types
encod_ans = processed_ans.copy()
object_type = categorical + remained
label_encoder = preprocessing.LabelEncoder()
for i in object_type:
    encod_ans[i] = label_encoder.fit_transform(encod_ans[i])

In [ ]:
# show boxplot of all columns
for i in encod_ans.columns:
    plt.boxplot(encod_ans[i])
    plt.title(i)
    plt.show()

In [ ]:
# features without outlier
no_outlier_columns = ['Respondent','CompFreq','Country','CurrencyDesc','CurrencySymbol','JobFactors',
                      'JobSat','JobSeek','LanguageDesireNextYear','LanguageWorkedWith','MiscTechWorkedWith',
                      'NEWCollabToolsWorkedWith','NEWDevOps','NEWEdImpt','NEWJobHuntResearch','NEWLearn',
                      'NEWOffTopic','NEWOnboardGood','NEWOtherComms','NEWOvertime','NEWPurpleLink',
                      'NEWStuck','OrgSize','PlatformDesireNextYear','PlatformWorkedWith','SOComm',
                      'SOVisitFreq','SurveyEase','YearsCode', 'YearsCodePro']

# discrete columns (no outlier)
discrete_columns = ['Hobbyist', 'Gender', 'PurchaseWhat', 'Sexuality', 'SOAccount', 'SurveyLength', 'Trans',
                    'UndergradMajor', 'WelcomeChange']

In [ ]:
# features with outlier
outlier_columns = list()
for i in encod_ans.columns:
    if i not in (no_outlier_columns + discrete_columns):
        outlier_columns.append(i)

In [ ]:
# detecting outliers
for i in outlier_columns:
    plt.plot(encod_ans[i], 'o')
    plt.title(i)
    plt.show()

In [ ]:
# outlier removal
for row in range(len(encod_ans['Age'])):
    if encod_ans.loc[row, 'Age'] > 250.0:
        remove = encod_ans.loc[row, 'Respondent']
encod_ans = encod_ans.set_index('Respondent').drop(remove)